In [14]:
import pandas as pd
import numpy as np
import glob
import nltk
import re
import string
from nltk.corpus import stopwords

In [ ]:
# 停用词 stopwords
stop_words = set(stopwords.words('english')) 

In [ ]:
# 读取csv文件
data_trump = pd.read_csv('data/US-Election-2020-Tweets/hashtag_donaldtrump.csv', lineterminator='\n')
data_biden = pd.read_csv('data/US-Election-2020-Tweets/hashtag_joebiden.csv', lineterminator='\n')

In [5]:
# csv文件合并
csv_list = glob.glob('data/US-Election-2020-Tweets/*.csv')
outputfile = 'data.csv'
print(u'共发现%s个CSV文件'% len(csv_list))
print(u'正在处理............')
def unite():
    for inputfile in csv_list:
        f=open(inputfile)
        data=pd.read_csv(f)
        data.to_csv(outputfile,mode='a',index=False)
    print('unite finish.')
unite()

共发现2个CSV文件
正在处理............


/Users/luhai/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2909: DtypeWarning: Columns (1,3,6,11,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


unite finish.


In [6]:
data1 = pd.read_csv('data.csv', lineterminator='\n')

/Users/luhai/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,3,4,6,11,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
# 去重
data2 = data1.drop_duplicates('tweet_id', keep='first')

In [11]:
data2

,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,...,user_followers_count,user_location,lat,long,city,country,continent,state,state_code,collected_at
0,2020-10-15 00:00:01,1.316529221557252e+18,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0,TweetDeck,360666534.0,El Sol Latino News,elsollatinonews,🌐 Noticias de interés para latinos de la costa...,...,1860.0,"Philadelphia, PA / Miami, FL",25.7743,-80.1937,NaN,United States of America,North America,Florida,FL,2020-10-21 00:00:00
1,2020-10-15 00:00:01,1.3165292227484303e+18,"Usa 2020, Trump contro Facebook e Twitter: cop...",26.0,9,Social Mediaset,331617619.0,Tgcom24,MediasetTgcom24,Profilo ufficiale di Tgcom24: tutte le notizie...,...,1067661.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-21 00:00:00.373216530
2,2020-10-15 00:00:02,1.316529228091847e+18,"#Trump: As a student I used to hear for years,...",2.0,1,Twitter Web App,8436472.0,snarke,snarke,"Will mock for food! Freelance writer, blogger,...",...,1185.0,Portland,45.5202,-122.674,Portland,United States of America,North America,Oregon,OR,2020-10-21 00:00:00.746433060
3,2020-10-15 00:00:02,1.316529227471237e+18,2 hours since last tweet from #Trump! Maybe he...,0.0,0,Trumpytweeter,8.28355589206057e+17,Trumpytweeter,trumpytweeter,"If he doesn't tweet for some time, should we b...",...,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-21 00:00:01.119649591
4,2020-10-15 00:00:08,1.3165292523014513e+18,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3,Twitter for iPhone,47413798.0,Rana Abtar - رنا أبتر,Ranaabtar,"Washington Correspondent, Lebanese-American ,c...",...,5393.0,Washington DC,38.895,-77.0366,Washington,United States of America,North America,District of Columbia,DC,2020-10-21 00:00:01.492866121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1748155,2020-11-08 23:59:37,1.3255888213088215e+18,Hypocrite!\n\n#Biden \n#Covid_19 https://t.co/...,2.0,0,Twitter Web App,9.58368e+17,van Lith de Jeude,LithJeude,"Stop this crazy and altruistic theory of ""We m...",...,541.0,Venus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-11-09 18:32:45.743523
1748157,2020-11-08 23:59:38,1.325588825654096e+18,Ωχ ελπίζω να μη μας βγει σαν τους οπαδούς του...,0.0,0,Twitter for Android,4.03282e+08,οχι άλλο κάρβουνο 🇬🇷🗣🗣🗣,anapodoi,ακραία καιρικά φαινόμενα... ζήσαμε και απόψε,...,772.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-11-09 18:32:45.947617
1748158,2020-11-08 23:59:41,1.3255888381237576e+18,L'OTAN va sortir de sa léthargie et redevenir ...,48.0,14,Twitter for Android,7.81918e+17,🇫🇷 Alt-Droite (matricule 6921) ✝️ 🇬🇷 🇮🇹 🇦🇲,CtrlAltDroite,Fils de mineur. Libertés - Identité - Solidari...,...,15806.0,France,46.6034,1.88833,NaN,France,Europe,NaN,NaN,2020-11-09 18:32:45.627335
1748159,2020-11-08 23:59:52,1.3255888813460152e+18,🌎\n\n“#congiuntifuoriregione”\n\n‘Sono felice ...,1.0,1,Twitter for iPhone,5.29332e+08,Angelo Tani,AngeloTani,nato a casa dei nonni,...,5974.0,🌎,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-11-09 18:32:45.599846


In [15]:
# tweet,user_description列分词，生成tweet_tokenize,user_tokenize列
x=re.compile('[%s]' % re.escape(string.punctuation))
data2['tweet'] = data2['tweet'].astype(str)
data2['user_description'] = data2['user_description'].astype(str)
def custom_tokenize(text):
    if not text:
        print('The text to be tokenized is a None type. Defaulting to blank string.')
        text = ''
    filtered_sentence = []
    for w in nltk.word_tokenize(text):
        new_w = x.sub(u'', w)  # 标点符号替换为空格
        if not new_w == u'' and new_w not in stop_words:  # 如果new_token不是空格，则加入new_review中，达到了去标点的效果
            filtered_sentence.append(new_w)
    return filtered_sentence
        
data2['tweet_tokenize'] = data2.tweet.apply(custom_tokenize)
data2['user_tokenize'] = data2.user_description.apply(custom_tokenize)

/Users/luhai/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/luhai/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/luhai/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [16]:
data2

,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,...,city,country,continent,state,state_code,collected_at,tweet_tokenize,user_tokenize,tweet_tokenize_stop,user_tokenize_stop
0,2020-10-15 00:00:01,1.316529221557252e+18,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0,TweetDeck,360666534.0,El Sol Latino News,elsollatinonews,🌐 Noticias de interés para latinos de la costa...,...,NaN,United States of America,North America,Florida,FL,2020-10-21 00:00:00,"[Elecciones2020, En, Florida, JoeBiden, dice, ...","[🌐, Noticias, de, interés, para, latinos, de, ...",#,🌐
1,2020-10-15 00:00:01,1.3165292227484303e+18,"Usa 2020, Trump contro Facebook e Twitter: cop...",26.0,9,Social Mediaset,331617619.0,Tgcom24,MediasetTgcom24,Profilo ufficiale di Tgcom24: tutte le notizie...,...,NaN,NaN,NaN,NaN,NaN,2020-10-21 00:00:00.373216530,"[Usa, 2020, Trump, contro, Facebook, e, Twitte...","[Profilo, ufficiale, di, Tgcom24, tutte, le, n...",Usa,Profilo
2,2020-10-15 00:00:02,1.316529228091847e+18,"#Trump: As a student I used to hear for years,...",2.0,1,Twitter Web App,8436472.0,snarke,snarke,"Will mock for food! Freelance writer, blogger,...",...,Portland,United States of America,North America,Oregon,OR,2020-10-21 00:00:00.746433060,"[Trump, As, student, I, used, hear, years, ten...","[Will, mock, food, Freelance, writer, blogger,...",#,Will
3,2020-10-15 00:00:02,1.316529227471237e+18,2 hours since last tweet from #Trump! Maybe he...,0.0,0,Trumpytweeter,8.28355589206057e+17,Trumpytweeter,trumpytweeter,"If he doesn't tweet for some time, should we b...",...,NaN,NaN,NaN,NaN,NaN,2020-10-21 00:00:01.119649591,"[2, hours, since, last, tweet, Trump, Maybe, V...","[If, nt, tweet, time, worried]",2,If
4,2020-10-15 00:00:08,1.3165292523014513e+18,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3,Twitter for iPhone,47413798.0,Rana Abtar - رنا أبتر,Ranaabtar,"Washington Correspondent, Lebanese-American ,c...",...,Washington,United States of America,North America,District of Columbia,DC,2020-10-21 00:00:01.492866121,"[You, get, tie, And, get, tie, Trump, ‘, rally...","[Washington, Correspondent, LebaneseAmerican, ...",You,Washington
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1748155,2020-11-08 23:59:37,1.3255888213088215e+18,Hypocrite!\n\n#Biden \n#Covid_19 https://t.co/...,2.0,0,Twitter Web App,9.58368e+17,van Lith de Jeude,LithJeude,"Stop this crazy and altruistic theory of ""We m...",...,NaN,NaN,NaN,NaN,NaN,2020-11-09 18:32:45.743523,"[Hypocrite, Biden, Covid19, https, tcof7v8vm4vbv]","[Stop, crazy, altruistic, theory, We, must, we...",Hypocrite,Stop
1748157,2020-11-08 23:59:38,1.325588825654096e+18,Ωχ ελπίζω να μη μας βγει σαν τους οπαδούς του...,0.0,0,Twitter for Android,4.03282e+08,οχι άλλο κάρβουνο 🇬🇷🗣🗣🗣,anapodoi,ακραία καιρικά φαινόμενα... ζήσαμε και απόψε,...,NaN,NaN,NaN,NaN,NaN,2020-11-09 18:32:45.947617,"[Ωχ, ελπίζω, να, μη, μας, βγει, σαν, τους, οπα...","[ακραία, καιρικά, φαινόμενα, ζήσαμε, και, απόψε]",Ωχ,ακραία
1748158,2020-11-08 23:59:41,1.3255888381237576e+18,L'OTAN va sortir de sa léthargie et redevenir ...,48.0,14,Twitter for Android,7.81918e+17,🇫🇷 Alt-Droite (matricule 6921) ✝️ 🇬🇷 🇮🇹 🇦🇲,CtrlAltDroite,Fils de mineur. Libertés - Identité - Solidari...,...,NaN,France,Europe,NaN,NaN,2020-11-09 18:32:45.627335,"[LOTAN, va, sortir, de, sa, léthargie, et, red...","[Fils, de, mineur, Libertés, Identité, Solidar...",L'OTAN,Fils
1748159,2020-11-08 23:59:52,1.3255888813460152e+18,🌎\n\n“#congiuntifuoriregione”\n\n‘Sono felice ...,1.0,1,Twitter for iPhone,5.29332e+08,Angelo Tani,AngeloTani,nato a casa dei nonni,...,NaN,NaN,NaN,NaN,NaN,2020-11-09 18:32:45.599846,"[🌎, “, congiuntifuoriregione, ”, ‘, Sono, feli...","[nato, casa, dei, nonni]",🌎,nato


In [18]:
# 删除tweet, user_description, user_location, state_code, collected_at列
data2 = data2.drop(['tweet', 'user_description', 'user_location', 'state_code', 'collected_at', 'tweet_tokenize_stop', 'user_tokenize_stop'], axis=1)

In [19]:
data2.astype

<bound method NDFrame.astype of                   created_at                tweet_id likes retweet_count  \
0        2020-10-15 00:00:01   1.316529221557252e+18   0.0             0   
1        2020-10-15 00:00:01  1.3165292227484303e+18  26.0             9   
2        2020-10-15 00:00:02   1.316529228091847e+18   2.0             1   
3        2020-10-15 00:00:02   1.316529227471237e+18   0.0             0   
4        2020-10-15 00:00:08  1.3165292523014513e+18   4.0             3   
...                      ...                     ...   ...           ...   
1748155  2020-11-08 23:59:37  1.3255888213088215e+18   2.0             0   
1748157  2020-11-08 23:59:38   1.325588825654096e+18   0.0             0   
1748158  2020-11-08 23:59:41  1.3255888381237576e+18  48.0            14   
1748159  2020-11-08 23:59:52  1.3255888813460152e+18   1.0             1   
1748160  2020-11-08 23:59:58  1.3255889087264236e+18   0.0             0   

                      source               user_id  \
0

In [20]:
data2.to_csv('data.csv', index=False, header=None)